# K8S Resource Test


In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP
SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

In [ ]:
import json
from collections import defaultdict
def testJson(model, iterations):
    cnts = defaultdict(int)
    %env model=$model
    for i in range(iterations):
        res = !curl -s http://${MESH_IP}/v2/models/${model}/infer -H "Content-Type: application/json" -H "seldon-model: ${model}"\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
        res = json.loads(res[0])
        cnts[res["model_name"]] += 1
    for name,cnt in cnts.items():
        print(name,cnt)
        
def testGrpc(model, iterations):
    cnts = defaultdict(int)
    %env model=$model
    for i in range(iterations):
        res = !grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
              -plaintext \
              -import-path ../../apis \
              -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
              -rpc-header "seldon-model: ${model}" \
              ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer
        res = json.loads(''.join(res))
        cnts[res["modelName"]] += 1       
    for name,cnt in cnts.items():
        print(name,cnt)
        
def testExperimentJson(experiment, iterations):
    cnts = defaultdict(int)
    %env model=$experiment
    for i in range(iterations):
        res = !curl -s http://${MESH_IP}:80/v2/models/exp1/infer -H "Content-Type: application/json" -H "seldon-model: ${model}.experiment"\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
        res = json.loads(res[0])
        cnts[res["model_name"]] += 1
    for name,cnt in cnts.items():
        print(name,cnt)
        
def testExperimentGrpc(experiment, iterations):
    cnts = defaultdict(int)
    %env model=$experiment
    for i in range(iterations):
        res = !grpcurl -d '{"model_name":"exp1","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
              -plaintext \
              -import-path ../../apis \
              -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
              -rpc-header "seldon-model:${model}.experiment" \
              ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer
        res = json.loads(''.join(res))
        cnts[res["modelName"]] += 1       
    for name,cnt in cnts.items():
        print(name,cnt)

## AB Test with iris default model

In [ ]:
!cat ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!cat ../../operator/samples/experiments/sklearn2.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/sklearn2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ../../operator/samples/experiments/ab-default-model.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/ab-default-model.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

In [ ]:
testJson("iris",50)

In [ ]:
testGrpc("iris",50)

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/ab-default-model.yaml

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/sklearn2.yaml

## Experiment over two iris models

In [ ]:
!cat ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!cat ../../operator/samples/experiments/sklearn2.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/sklearn2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ../../operator/samples/experiments/ab.yaml

In [ ]:
!kubectl create -f ../../operator/samples/experiments/ab.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

In [ ]:
testJson("iris",50)

In [ ]:
testExperimentJson("experiment-iris", 50)

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/ab.yaml

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/sklearn1.yaml

In [ ]:
!kubectl delete -f ../../operator/samples/experiments/sklearn2.yaml